In [1]:
import torch
import numpy as np
import cv2
import time
import datetime
import pandas as pd
from PIL import Image
from yolov5.models.experimental import attempt_load
import warnings
warnings.filterwarnings(action = 'ignore')

from convert_data import collect_data, collect_coordinates
from class_mapping import yolo_class_mapping, fire_class_mapping, animals_class_mapping, obstacle_class_mapping
from detection import category
from perform_object import perform_object
from yolo_to_server import send_server

WARNING ⚠️ 'ultralytics.yolo.v8' is deprecated since '8.0.136' and will be removed in '8.1.0'. Please use 'ultralytics.models.yolo' instead.
WARNING ⚠️ 'ultralytics.yolo.utils' is deprecated since '8.0.136' and will be removed in '8.1.0'. Please use 'ultralytics.utils' instead.
Note this warning may be related to loading older models. You can update your model to current structure with:
    import torch
    ckpt = torch.load("model.pt")  # applies to both official and custom models
    torch.save(ckpt, "updated-model.pt")



#### Version check

In [2]:
print('opencv version: ', cv2.__version__)

opencv version:  4.8.0


In [3]:
camera_name = 'KFQ StarValley'
device = torch.device('cpu')

# Load trained YOLOv5 model weights
yolo_weights_path = '/Users/bongeungu/Desktop/kfq/KFQ_TEAM01/yolo/models/yolov5m.pt'
fire_weights_path = '/Users/bongeungu/Desktop/kfq/KFQ_TEAM01/yolo/models/fire_smoke.pt'
animals_weights_path = '/Users/bongeungu/Desktop/kfq/KFQ_TEAM01/yolo/models/animals.pt'
obstacle_weights_path = '/Users/bongeungu/Desktop/kfq/KFQ_TEAM01/yolo/models/obstacle.pt'

yolo_model = attempt_load(yolo_weights_path, device=device)
fire_model = attempt_load(fire_weights_path, device=device)
animals_model = attempt_load(animals_weights_path, device=device)
obstacle_model = attempt_load(obstacle_weights_path, device=device)

Fusing layers... 
YOLOv5m summary: 369 layers, 21190557 parameters, 0 gradients
Fusing layers... 
YOLOv5m summary: 369 layers, 20879400 parameters, 0 gradients, 48.2 GFLOPs
Fusing layers... 
YOLOv5m summary: 369 layers, 20887482 parameters, 0 gradients, 48.3 GFLOPs
Fusing layers... 
YOLOv5m summary: 369 layers, 20887482 parameters, 0 gradients, 48.3 GFLOPs


In [4]:
# Initialize webcam
cap = cv2.VideoCapture(0)

columns = ['X1', 'Y1', 'X2', 'Y2', 'Class_Name']
road_coordinates = pd.DataFrame(columns=columns)

for i in range(200):
    ret, frame = cap.read()
    
    image = Image.fromarray(frame)
    image = image.resize((640,640))
    image_tensor = torch.from_numpy(np.array(image)).float().permute(2, 0, 1) / 255.0
    image_tensor = image_tensor.unsqueeze(0).to(device)
    
    results = perform_object(image_tensor, 0.65, yolo_model)
    road_coordinates = collect_coordinates(results, frame, yolo_class_mapping, road_coordinates)
    
road_coordinates = road_coordinates.sort_values(by = ['X1', 'Y1'])
print(road_coordinates)

       X1   Y1    X2   Y2 Class_Name
161  1214  529  1370  646        car
144  1394  540  1680  680        car
143  1405  541  1710  683        car
141  1441  540  1760  680        car
139  1465  542  1777  681        car
..    ...  ...   ...  ...        ...
100  1827  570  1919  680        car
18   1837  309  1918  453        bus
1    1845  327  1920  494        bus
98   1847  579  1920  678        car
97   1869  589  1919  675        car

[218 rows x 5 columns]


In [5]:
print(len(road_coordinates))

218


In [6]:
# Initialize webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()  # Read frame from the webcam
    captured_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S") # 상황 발생 시간 기록
    print(captured_time)
    
    # reset dataframe
    columns = ['Time', 'Class_Name', 'Confidence', 'X1', 'Y1', 'X2', 'Y2', 'action_detection', 'action_category', 'event_type']
    results_df = pd.DataFrame(columns=columns)
    
    # Preprocess the frame
    image = Image.fromarray(frame)
    image = image.resize((640, 640))  # Resize the image to the model's input size
    image_tensor = torch.from_numpy(np.array(image)).float().permute(2, 0, 1) / 255.0
    image_tensor = image_tensor.unsqueeze(0).to(device)

    # Perform object detection
    results = perform_object(image_tensor, 0.65, yolo_model)
    results_df, road_coordinates = collect_data(results, frame, yolo_class_mapping, results_df, captured_time, road_coordinates)
    print(results_df)
    
    results = perform_object(image_tensor, 0.3, fire_model)
    results_df, road_coordinates = collect_data(results, frame, fire_class_mapping, results_df, captured_time, road_coordinates)
    
    results = perform_object(image_tensor, 0.65, animals_model)
    results_df, road_coordinates = collect_data(results, frame, animals_class_mapping, results_df, captured_time, road_coordinates)
    
    results = perform_object(image_tensor, 0.4, obstacle_model)
    results_df, road_coordinates = collect_data(results, frame, obstacle_class_mapping, results_df, captured_time, road_coordinates)
    
    print(road_coordinates)
    
    results_df = category(results_df, road_coordinates)

    # Display the frame with bounding boxes and labels
    cv2.imshow(camera_name, frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):  # Press 'q' to exit the loop
        break
    
    send_server(results_df, camera_name)
    
    time.sleep(2) # 2초 간격으로 사진 캡쳐(30 frames per 1 miniute)

# Save the DataFrame to a CSV file
results_df.to_csv('object_detection_results.csv', index=False)

# Release the webcam and close windows
cap.release()
cv2.destroyAllWindows()

2023-08-22 15:38:46
                  Time Class_Name       Confidence    X1   Y1    X2    Y2  \
0  2023-08-22 15:38:46     person  tensor(0.86002)  1367  889  1477  1079   
1  2023-08-22 15:38:46     person  tensor(0.83182)  1730  654  1800   806   
2  2023-08-22 15:38:46     person  tensor(0.80584)  1340  712  1423   947   
3  2023-08-22 15:38:46     person  tensor(0.80089)  1463  618  1541   811   
4  2023-08-22 15:38:46        bus  tensor(0.70899)  1566  330  1858   511   
5  2023-08-22 15:38:46     person  tensor(0.68299)  1021  996  1108  1080   
6  2023-08-22 15:38:46     person  tensor(0.65187)  1143  967  1277  1080   

   action_detection action_category event_type  
0                 0             NaN        NaN  
1                 0             NaN        NaN  
2                 0             NaN        NaN  
3                 0             NaN        NaN  
4                 0             NaN        NaN  
5                 0             NaN        NaN  
6                 0 

2023-08-22 15:39:01.481 python[93000:5998580] IMKClient Stall detected, *please Report* your user scenario attaching a spindump (or sysdiagnose) that captures the problem - (imkxpc_bundleIdentifierWithReply:) block performed very slowly (2.73 secs).


2023-08-22 15:39:03
                  Time Class_Name       Confidence    X1   Y1    X2   Y2  \
0  2023-08-22 15:39:03        bus  tensor(0.84974)  1565  330  1856  508   
1  2023-08-22 15:39:03     person  tensor(0.69546)  1751  472  1813  585   
2  2023-08-22 15:39:03        car  tensor(0.68229)  1379  367  1525  442   

   action_detection action_category event_type  
0                 0             NaN        NaN  
1                 0             NaN        NaN  
2                 0             NaN        NaN  
       X1   Y1    X2   Y2 Class_Name
0    1214  529  1370  646        car
1    1364  364  1525  443        car
222  1379  367  1525  442        car
2    1394  540  1680  680        car
3    1398  368  1525  441        car
..    ...  ...   ...  ...        ...
217  1827  570  1919  680        car
218  1837  309  1918  453        bus
219  1845  327  1920  494        bus
220  1847  579  1920  678        car
221  1869  589  1919  675        car

[223 rows x 5 columns]
2023-08-22 

In [9]:
print(len(road_coordinates))

49392


In [10]:
road_coordinates

,X1,Y1,X2,Y2
13,-4,358,1410,897
5,0,341,404,912
20,0,503,468,690
56,0,529,166,717
21,1,502,326,683
...,...,...,...,...
49387,1242,313,1916,606
49388,1242,313,1916,606
49389,1242,313,1916,606
49390,1242,313,1916,606
